In [1]:
import numpy as np # linear algebra
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
%matplotlib inline

The data files are

RegularSeasonCompactResults.csv

RegularSeasonDetailedResults.csv

Seasons.csv

Teams.csv

TourneyCompactResults.csv

TourneyDetailedResults.csv

TourneySeeds.csv

TourneySlots.csv

sample_submission.csv

# Loading all data files

In [2]:
df_seeds = pd.read_csv('TourneySeeds.csv')
df_tour_results = pd.read_csv('TourneyCompactResults.csv')
df_reg_comp_rs=pd.read_csv('RegularSeasonCompactResults.csv')
df_reg_det_rs=pd.read_csv('RegularSeasonDetailedResults.csv')
df_seasons=pd.read_csv('Seasons.csv')
df_teams=pd.read_csv('Teams.csv')
df_tour_det_results = pd.read_csv('TourneyDetailedResults.csv')
df_tour_slots = pd.read_csv('TourneySlots.csv')

In [3]:
print(df_seeds.shape)
print(df_seeds[df_seeds.Season==2016].shape)
df_seeds.head()

(2082, 3)
(68, 3)


,Season,Seed,Team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [4]:
df_tour_results.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [5]:
print(df_reg_comp_rs.shape)
df_reg_comp_rs.tail()

(145289, 8)


,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
145284,2016,132,1114,70,1419,50,N,0
145285,2016,132,1163,72,1272,58,N,0
145286,2016,132,1246,82,1401,77,N,1
145287,2016,132,1277,66,1345,62,N,0
145288,2016,132,1386,87,1433,74,N,0


In [6]:
df_seasons.head()

,Season,Dayzero,Regionw,Regionx,Regiony,Regionz
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [7]:
print(df_teams.shape)
df_teams.head()

(364, 2)


,Team_Id,Team_Name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [8]:
df_tour_det_results.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [9]:
print(df_tour_results.shape)
print(df_tour_results.columns)
print(df_tour_det_results.shape)
print(df_tour_det_results.columns)

(2050, 8)
Index(['Season', 'Daynum', 'Wteam', 'Wscore', 'Lteam', 'Lscore', 'Wloc',
       'Numot'],
      dtype='object')
(914, 34)
Index(['Season', 'Daynum', 'Wteam', 'Wscore', 'Lteam', 'Lscore', 'Wloc',
       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr',
       'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
       'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf'],
      dtype='object')


# Quick Analysis of Tourney Results and Regular Season Results


In [10]:
# Tourney Results
print(df_tour_results['Season'].unique())
print(df_tour_det_results['Season'].unique())
print(df_reg_det_rs['Season'].unique())
print(df_reg_comp_rs['Season'].unique())


[1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999
 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016]
[2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016]
[2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016]
[1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999
 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016]


The above result shows that compact results are available since 1985 but detailed results are only available since 2003

In [11]:
# Analyzing tour detail results and regular season results

print(df_tour_det_results.shape)
print(len(df_tour_det_results[df_tour_det_results['Season']==2016]))

(914, 34)
67


In [12]:
df_tour_det_results.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
909,2016,146,1314,88,1323,74,N,0,32,52,...,18,11,14,5,10,9,7,3,1,18
910,2016,146,1393,68,1438,62,N,0,21,57,...,21,10,12,8,26,21,13,3,8,16
911,2016,152,1314,83,1393,66,N,0,35,65,...,25,4,13,16,15,5,10,8,4,15
912,2016,152,1437,95,1328,51,N,0,35,49,...,27,7,11,19,10,8,17,6,2,20
913,2016,154,1437,77,1314,74,N,0,28,48,...,17,9,13,16,20,16,11,5,3,20


In [13]:
print(df_teams[df_teams['Team_Id']==1437])
print(df_teams[df_teams['Team_Id']==1314])
print(df_seeds[(df_seeds['Season']==2016) & ((df_seeds['Team']==1437) | (df_seeds['Team']==1314))])


     Team_Id  Team_Name
336     1437  Villanova
     Team_Id       Team_Name
213     1314  North Carolina
      Season Seed  Team
2014    2016  W01  1314
2049    2016  Y02  1437


# Villanova was the winner of 2016 Tournament, they were seeded 2nd in South, North Carolina was seeded 1st in the East

In [14]:
df_2016_seeds=df_seeds[df_seeds['Season']==2016]

In [15]:
print(df_2016_seeds)

      Season  Seed  Team
2014    2016   W01  1314
2015    2016   W02  1462
2016    2016   W03  1452
2017    2016   W04  1246
2018    2016   W05  1231
2019    2016   W06  1323
2020    2016   W07  1458
2021    2016   W08  1425
2022    2016   W09  1344
2023    2016   W10  1338
2024    2016  W11a  1276
2025    2016  W11b  1409
2026    2016   W12  1151
2027    2016   W13  1392
2028    2016   W14  1372
2029    2016   W15  1451
2030    2016  W16a  1192
2031    2016  W16b  1195
2032    2016   X01  1438
2033    2016   X02  1277
2034    2016   X03  1428
2035    2016   X04  1235
2036    2016   X05  1345
2037    2016   X06  1371
2038    2016   X07  1173
2039    2016   X08  1403
2040    2016   X09  1139
2041    2016   X10  1393
2042    2016   X11  1211
2043    2016   X12  1114
...      ...   ...   ...
2052    2016   Y05  1268
2053    2016   Y06  1112
2054    2016   Y07  1234
2055    2016   Y08  1160
2056    2016   Y09  1163
2057    2016   Y10  1396
2058    2016  Y11a  1435
2059    2016  Y11b  1455


The above result shows that the seeds are not a great feature because there are 4 different brackets, but each team plays in its own bracket first so sees are ok in that case

# Analyzing total number of wins of Villanova and North Carolina from 2016 Regular Season

In [16]:
df_reg_det_rs.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [17]:
Villanova_filter=(df_reg_det_rs['Season']==2016) & ((df_reg_det_rs['Wteam']==1437) | (df_reg_det_rs['Lteam']==1437))
df_2016_villanova=df_reg_det_rs[Villanova_filter]
print(df_2016_villanova.shape)

nc_filter=(df_reg_det_rs['Season']==2016) & ((df_reg_det_rs['Wteam']==1314) | (df_reg_det_rs['Lteam']==1314))
df_2016_nc=df_reg_det_rs[nc_filter]
print(df_2016_nc.shape)

(34, 34)
(34, 34)


In [18]:
df_2016_villanova[df_2016_villanova.Lteam==1437]

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
67103,2016,35,1328,78,1437,55,N,0,28,60,...,32,11,16,9,24,8,9,7,3,14
67533,2016,47,1438,86,1437,75,H,0,26,46,...,26,13,17,10,9,17,10,6,1,24
69001,2016,82,1344,82,1437,76,A,1,30,65,...,31,13,18,11,21,21,16,13,4,17
70496,2016,114,1462,90,1437,83,H,0,30,60,...,31,11,13,9,24,15,10,5,3,25
71227,2016,131,1371,69,1437,67,N,0,27,55,...,23,11,15,10,25,10,12,5,3,18


# Building a df with Regular Season Win% of each team by Season



In [19]:
years = df_reg_comp_rs['Season'].unique().tolist()
years

[1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016]

In [20]:
years = df_reg_comp_rs['Season'].unique().tolist()
rows_list = []
for year in years:
    teams = df_seeds['Team'][df_seeds['Season']==year].tolist()
    print(year)
    for t in teams:
        wins = len(df_reg_comp_rs[(df_reg_comp_rs['Season']==year) & (df_reg_comp_rs['Wteam']==t)])
        losses = len(df_reg_comp_rs[(df_reg_comp_rs['Season']==year) & (df_reg_comp_rs['Lteam']==t)])
        #print(y,t,wins/(wins+losses))
        dict1={'Season':year,'Team':t,'win_percent':wins/(wins+losses)}
        rows_list.append(dict1)

df_wins = pd.DataFrame(rows_list,columns=['Season','Team','win_percent'])

1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [21]:
print(df_wins.shape)
df_wins.head()

(2082, 3)


,Season,Team,win_percent
0,1985,1207,0.925926
1,1985,1210,0.774194
2,1985,1228,0.741935
3,1985,1260,0.833333
4,1985,1374,0.733333


In [22]:
#Testing df_wins

df_wins[(df_wins['Season']==2016) & (df_wins['Team']==1437)]
#print(df_wins.tail())

,Season,Team,win_percent
2049,2016,1437,0.852941


In [23]:
len(df_2016_villanova[df_2016_villanova.Wteam==1437])/len(df_2016_villanova)  

0.8529411764705882

The above two number matches that means df_wins is good

In [24]:
df_wins[(df_wins['Season']==2016) & (df_wins['Team']==1314)]


,Season,Team,win_percent
2014,2016,1314,0.823529


In [25]:
len(df_wins[(df_wins['Season']==2013)] )

68

In [28]:
len(df_wins[(df_wins['Season']==2016)] )

68

In [29]:
len(df_wins['Team'].unique() )

292

In [32]:
len(df_reg_comp_rs[df_reg_comp_rs['Season']==2016]['Wteam'].unique())

351